## Analysis ScanCount for Disruptions

Mode

1 - Bus

2 - Train

3 - Tram

In [1]:
import pandas as pd
import os

print(pd.__version__)

0.23.0


In [2]:
def load_data(the_query, the_file_dir, force_dl = False):
    if not(os.path.isfile(the_file_dir)) or force_dl :
        print('Loading data from BigQuery')
        the_df = pd.read_gbq(the_query,
                             project_id='republic-of-data-2018',
                             private_key="../MelbDatathon2018/Auth/Republic of Data 2018-3f4dcd98a279.json",
                             dialect='standard'
                            )
        the_df.to_hdf(the_file_dir, key='df', mode='w')
    else:
        print('Loading data from local')
        the_df = pd.read_hdf(the_file_dir)
    return(the_df)

In [3]:
query = """
SELECT
    *
FROM `analytics.Top_Stop_WeekdayPeak_ScanOnCount`
"""

file_name = '../MelbDatathon2018/Data/Top_Stop_WeekdayPeak_ScanOnCount.h5'

df_topstop_scancount = load_data(query, file_name, force_dl=False)

Loading data from local


In [4]:
query = """
SELECT
    Date, DayTypeCategory
FROM `data.calendar`
WHERE DayTypeCategory = 'Normal Weekday'
ORDER BY Date
"""

file_name = '../MelbDatathon2018/Data/calendar.h5'

df_calendar = load_data(query, file_name, force_dl=False)
df_calendar['Date'] = pd.to_datetime(df_calendar.Date)

Loading data from local


In [5]:
df_topstop_scancount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53659 entries, 0 to 53658
Data columns (total 4 columns):
StopID          53659 non-null int64
BusinessDate    53659 non-null datetime64[ns]
ScanOnCount     53659 non-null int64
ScanOffCount    53659 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 2.0 MB


In [6]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3776 entries, 0 to 3775
Data columns (total 2 columns):
Date               3776 non-null datetime64[ns]
DayTypeCategory    3776 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 88.5+ KB


In [7]:
df_merge = pd.merge(df_topstop_scancount, df_calendar,  how='inner', left_on=['BusinessDate'], right_on = ['Date'])

In [8]:
list_stopid = df_merge.StopID.unique()

In [9]:
def getIQR_upper(series):
    return((series.describe()[6] - series.describe()[4]) * 3 + series.describe()[6])

result = []

for stopid in df_merge.StopID.unique():
    df_temp = df_merge[df_merge.StopID == stopid].sort_values('BusinessDate')
    df_temp["ScanOnCount_Nextday"] = df_temp.shift(-1).ScanOnCount
    df_temp["Change_pec"] = ((df_temp.ScanOnCount_Nextday - df_temp.ScanOnCount)* 1.0)/df_temp.ScanOnCount
    ma_limit = getIQR_upper(df_temp.Change_pec.rolling(20).mean())
    df_temp['moving_avg'] = df_temp.Change_pec.rolling(20).mean().abs().fillna(0)
    result.append(df_temp[df_temp.moving_avg > (ma_limit if ma_limit > 0.05 else 0.05)])

In [10]:
pd.concat(result).shape

(2038, 9)

In [11]:
pd.concat(result).sample(5)

,StopID,BusinessDate,ScanOnCount,ScanOffCount,Date,DayTypeCategory,ScanOnCount_Nextday,Change_pec,moving_avg
11249,20004,2016-07-26,1857,138,2016-07-26,Normal Weekday,1882.0,0.013463,0.056701
27179,19891,2018-03-13,1807,75,2018-03-13,Normal Weekday,1737.0,-0.038738,33.017226
22543,19984,2018-04-26,401,81,2018-04-26,Normal Weekday,393.0,-0.019950,0.056325
6617,19888,2016-08-30,2461,1992,2016-08-30,Normal Weekday,2596.0,0.054856,0.051212
14436,19900,2017-02-23,2899,356,2017-02-23,Normal Weekday,2655.0,-0.084167,114.205475


In [ ]:
pd.concat(result).to_csv('output/potential_disruption_by_scancount_csv')